In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.offline as pyo
import plotly.express as px
pyo.init_notebook_mode(connected=True)
import utils as u

In [3]:
"""
Plain old dataset:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)

Enhanced with impressions:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)
"""

'\nPlain old dataset:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n\nEnhanced with impressions:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n'

In [4]:
"""
These destinations are selected based on their query pool sizes, and how representative
they are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)

9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),
482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),
9D883ED995259EA3E715122397FE0EF9 has query pool of size 2
"""
# selected_destinations = ["9FF55", "482E9", "9D883"]

'\nThese destinations are selected based on their query pool sizes, and how representative\nthey are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)\n\n9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),\n482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),\n9D883ED995259EA3E715122397FE0EF9 has query pool of size 2\n'

In [5]:

# def extra_df_prep(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.assign(
#         destination=df.apply(
#             lambda row: row.destination[0:5],
#             axis=1,
#         )
#     )
#     global selected_destinations
#     return df[df.destination.isin(selected_destinations)]


In [6]:
# unaugmented path
# path = "ray/criteo/bias_varying_epoch_size"

In [7]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [8]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [9]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [10]:
# u.save_data(path)

In [11]:
# augmented impressions path
# path = "ray/criteo/augmented_bias_varying_epoch_size"

In [12]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [13]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [14]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [18]:
# augmented convesrions path
path = "ray/criteo/augmented_conversions_bias_varying_epoch_size/run_and_report_2024-04-30_15-24-35"

In [16]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [19]:
u.save_data(path)

In [29]:
original_conversions = pd.read_csv("../data/criteo/criteo_query_pool_conversions.csv")
original_conversions = original_conversions.groupby(["partner_id", "user_id", "product_category3"])["count"].count()
multi_convert_percentage = original_conversions[original_conversions > 1].shape[0] / original_conversions.shape[0] * 100
print(f"{multi_convert_percentage:.5f}% of users convert more than once")

C:\Users\petem\AppData\Local\Temp\ipykernel_8636\293685438.py:1: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



1.07556% of users convert more than once


In [30]:
augmented_conversions_10 = pd.read_csv("../data/criteo/criteo_query_pool_augmented_conversions_10.csv")
augmented_conversions_10 = augmented_conversions_10.groupby(["partner_id", "user_id", "product_category3"])["count"].count()
multi_convert_percentage = augmented_conversions_10[augmented_conversions_10 > 1].shape[0] / augmented_conversions_10.shape[0] * 100
print(f"{multi_convert_percentage:.5f}% of users convert more than once")

11.64880% of users convert more than once


In [32]:
augmented_conversions_30 = pd.read_csv("../data/criteo/criteo_query_pool_augmented_conversions_30.csv")
augmented_conversions_30 = augmented_conversions_30.groupby(["partner_id", "user_id", "product_category3"])["count"].count()
multi_convert_percentage = augmented_conversions_30[augmented_conversions_30 > 1].shape[0] / augmented_conversions_30.shape[0] * 100
print(f"{multi_convert_percentage:.5f}% of users convert more than once")

31.97529% of users convert more than once


In [34]:
augmented_conversions_50 = pd.read_csv("../data/criteo/criteo_query_pool_augmented_conversions.csv")
augmented_conversions_50 = augmented_conversions_50.groupby(["partner_id", "user_id", "product_category3"])["count"].count()
multi_convert_percentage = augmented_conversions_50[augmented_conversions_50 > 1].shape[0] / augmented_conversions_50.shape[0] * 100
print(f"{multi_convert_percentage:.5f}% of users convert more than once")

48.80594% of users convert more than once
